In [1]:
import os
import mlflow
import requests
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_diabetes

In [4]:
import psycopg2
import pandas as pd

# Conexión a la base de datos PostgreSQL
def get_postgres_connection():
    conn = psycopg2.connect(
        host='10.43.101.202',         # Nombre del servicio de PostgreSQL en Docker
        port='5432',             # Puerto predeterminado de PostgreSQL
        user='airflow',          # Usuario de PostgreSQL
        password='airflow',      # Contraseña de PostgreSQL
        database='airflow'    # Nombre de la base de datos
    )
    return conn

# Query the covertype table and return a DataFrame
def query_covertype():
    query = "SELECT * FROM covertype"
    conn = get_postgres_connection()
    
    # Use pandas to execute the query and store the result in a DataFrame
    df = pd.read_sql(query, conn)
    
    # Close the connection
    conn.close()

    # Return the DataFrame
    return df

# Query the table and store the result in a DataFrame
df_covertype = query_covertype()

# Display the DataFrame
print(df_covertype.head())  # Show the first few rows of the DataFrame


/tmp/ipykernel_3871698/199442818.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


DatabaseError: Execution failed on sql 'SELECT * FROM covertype': relation "covertype" does not exist
LINE 1: SELECT * FROM covertype
                      ^


In [ ]:
# Load the dataset to a dataframe
df = data

# Set the target values
y = df['Cover_Type']#.values

# Set the input values
df.drop('Cover_Type', axis=1, inplace=True)
X = df#.values

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore'),
                                        ["Wilderness_Area", "Soil_Type"]),
                                      remainder='passthrough') # pass all the numeric values through the pipeline without any changes.

column_trans

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Wilderness_Area', 'Soil_Type'])])

In [9]:
pipe = Pipeline(steps=[("column_trans", column_trans),("scaler", StandardScaler(with_mean=False)), ("RandomForestClassifier", RandomForestClassifier())])

pipe

Pipeline(steps=[('column_trans',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Wilderness_Area',
                                                   'Soil_Type'])])),
                ('scaler', StandardScaler(with_mean=False)),
                ('RandomForestClassifier', RandomForestClassifier())])

In [10]:
param_grid =  {'RandomForestClassifier__max_depth': [1,2,3,10], 'RandomForestClassifier__n_estimators': [10,11]}

search = GridSearchCV(pipe, param_grid, n_jobs=2)
search


GridSearchCV(estimator=Pipeline(steps=[('column_trans',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Wilderness_Area',
                                                                          'Soil_Type'])])),
                                       ('scaler',
                                        StandardScaler(with_mean=False)),
                                       ('RandomForestClassifier',
                                        RandomForestClassifier())]),
             n_jobs=2,
             param_grid={'RandomForestClassifier__max_depth': [1, 2, 3, 10],
                         'RandomForestClassifier__n_estimators': [10, 11]})

In [11]:
search.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__column_trans', 'estimator__scaler', 'estimator__RandomForestClassifier', 'estimator__column_trans__n_jobs', 'estimator__column_trans__remainder', 'estimator__column_trans__sparse_threshold', 'estimator__column_trans__transformer_weights', 'estimator__column_trans__transformers', 'estimator__column_trans__verbose', 'estimator__column_trans__verbose_feature_names_out', 'estimator__column_trans__onehotencoder', 'estimator__column_trans__onehotencoder__categories', 'estimator__column_trans__onehotencoder__drop', 'estimator__column_trans__onehotencoder__dtype', 'estimator__column_trans__onehotencoder__handle_unknown', 'estimator__column_trans__onehotencoder__max_categories', 'estimator__column_trans__onehotencoder__min_frequency', 'estimator__column_trans__onehotencoder__sparse', 'estimator__column_trans__onehotencoder__sparse_output', 'estimator__scaler__copy', 'estimator__scaler__wit

In [29]:
!pip show scikit-learn
!pip show mlflow

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.9/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: category-encoders, imbalanced-learn, mlflow, pmdarima, pycaret, pyod, scikit-plot, sktime, tbats, yellowbrick
Name: mlflow
Version: 2.3.0
Summary: MLflow: A Platform for ML Development and Productionization
Home-page: https://mlflow.org/
Author: Databricks
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.9/site-packages
Requires: alembic, click, cloudpickle, databricks-cli, docker, entrypoints, Flask, gitpython, gunicorn, importlib-metadata, Jinja2, markdown, matplotlib, numpy, packaging, pandas, protobuf, pyarrow, pytz, pyyaml, querystring-parser, requests, scikit-learn, scipy, sqlalchemy, sqlparse
Required-by: 


In [12]:
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://10.43.101.175:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

# connect to mlflow
mlflow.set_tracking_uri("http://10.43.101.175:5000")
mlflow.set_experiment("mlflow_tracking_examples")

mlflow.sklearn.autolog(log_model_signatures=True, log_input_examples=True, registered_model_name="modelo1")

with mlflow.start_run(run_name="autolog_pipe_model_reg") as run:
    search.fit(X_train, y_train)

2025/03/14 17:56:00 WARNING mlflow.utils: Truncated the value of the key `estimator`. Truncated value: `Pipeline(steps=[('column_trans',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Wilderness_Area',
                                                   'Soil_Type'])])),
                ('scaler', StandardScaler(with_mean=False)),
                ('RandomForestClassifier', Rand...`
2025/03/14 17:56:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.9/site-packages/mlflow/models/signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and wil

In [4]:
#EXPERIMIENTOS

In [28]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
import os
import random

os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://10.43.101.175:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

mlflow.set_tracking_uri("http://10.43.101.175:5000")
mlflow.set_experiment("mlflow_tracking_examples")

mlflow.sklearn.autolog(log_model_signatures=True, log_input_examples=True, registered_model_name="modelo1")

# Cambiar los hiperparámetros cada vez que se ejecute
n_estimators = random.randint(50, 200)
max_depth = random.randint(3, 10)
max_features = random.choice([2, 3, 4, 'auto'])

with mlflow.start_run(run_name="autolog_pipe_model_reg") as run:
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
    rf.fit(X_train, y_train)
    mlflow.log_metric("r2_score", rf.score(X_test, y_test))

print("Experimento registrado correctamente.")


/usr/local/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
Registered model 'modelo1' already exists. Creating a new version of this model...
2025/03/14 18:00:09 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: modelo1, version 20
Created version '20' of model 'modelo1'.


Experimento registrado correctamente.
